# PROOF Plots in 2D

## KMeans

In [38]:
import cluster_data
from cluster_data import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
from kmeans import k_means
from cluster_plotter import ClusterPlotter
from clustering_utils import ClusterData
import cluster_plotter

IAC_range = {"2005-2008": np.arange(2005, 2009)}

binned_data = cluster_data.bin_data_for_clustering(IAC_range, print_res=False)

k_values = [5, 6, 7, 9]

results_per_year_range = {}

# Output directory for plots
plot_dir = "Images\IAC_plots\KMeans_2d"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:  
    print(f"\nRunning K-Means for Year Range: {year_range}")

    data_array = np.array([cluster_data.inc, cluster_data.raan]).T
    normalized_data, data_min, data_max = normalize_data(data_array)

    for k in k_values:
        result_kmeans, _, _, _, _ = run_clustering(
            k_means, f"K-means (k={k})", normalized_data, data_min, data_max, k, plot=False, init='kmeans++'
        )

        unnormalized_data, cluster_centers = unnormalize(
            result_kmeans.data, result_kmeans.cluster_centers, data_min, data_max
        )

        plotter = ClusterPlotter(unnormalized_data, result_kmeans.labels, cluster_centers)
        plot_filename = os.path.join(plot_dir, f"kmeans_{year_range}_k{k}.png")
        title = f"k-Means: years = {year_range}, k = {k}"
        plotter.clusters_2d_plot(title, plot_filename)


<>:20: SyntaxWarning: invalid escape sequence '\I'
<>:20: SyntaxWarning: invalid escape sequence '\I'
C:\Users\fionu\AppData\Local\Temp\ipykernel_23496\3540405002.py:20: SyntaxWarning: invalid escape sequence '\I'
  plot_dir = "Images\IAC_plots\KMeans_2d"


{'2005-2008': {'geo': ['../input/stat_Master_05_geo_s1.crs', '../input/stat_Master_06_geo_s1.crs', '../input/stat_Master_07_geo_s1.crs', '../input/stat_Master_08_geo_s1.crs'], 'gto': ['../input/stat_Master_05_gto_s1.crs', '../input/stat_Master_06_gto_s1.crs', '../input/stat_Master_07_gto_s1.crs', '../input/stat_Master_08_gto_s1.crs'], 'fol': ['../input/stat_Master_05_fol_s1.crs', '../input/stat_Master_06_fol_s1.crs', '../input/stat_Master_07_fol_s1.crs', '../input/stat_Master_08_fol_s1.crs']}}

Running K-Means for Year Range: 2005-2008
Runtime for k_means: 0.012234 seconds
Plot saved as: Images\IAC_plots\KMeans_2d\kmeans_2005-2008_k5.png
Runtime for k_means: 0.038904 seconds
Plot saved as: Images\IAC_plots\KMeans_2d\kmeans_2005-2008_k6.png
Runtime for k_means: 0.082343 seconds
Plot saved as: Images\IAC_plots\KMeans_2d\kmeans_2005-2008_k7.png
Runtime for k_means: 0.290182 seconds
Plot saved as: Images\IAC_plots\KMeans_2d\kmeans_2005-2008_k9.png


## DBSCAN

In [40]:
import cluster_data
from cluster_data import run_clustering, normalize_data, unnormalize
import numpy as np
import pandas as pd
import os
import cluster_plotter
from DBSCAN import dbscan_clustering
from cluster_plotter import ClusterPlotter
from clustering_utils import ClusterData

# Define year ranges
IAC_range = {"2005-2009": np.arange(2005, 2010)}

binned_data = cluster_data.bin_data_for_clustering(IAC_range, print_res=False)

results_per_year_range = {}

# Output directory for plots
plot_dir = "Images\IAC_plots\DBSCAN_2d_5years"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

for cluster_data, year_range in binned_data:  
    print(f"\nRunning DBSCAN for Year Range: {year_range}")

    data_array = np.array([cluster_data.inc, cluster_data.raan]).T
    normalized_data, data_min, data_max = normalize_data(data_array)

    # DBSCAN parameter ranges
    #eps_values = [0.02, 0.01, 0.015]
    #min_samples_values = [10, 15, 20, 25, 30]

    eps_values = [0.015]
    min_samples_values = [27]

    for eps in eps_values:
        for min_samples in min_samples_values:
            result_dbscan, _, _, _, _ = run_clustering(
                dbscan_clustering, "DBSCAN", normalized_data, data_min, data_max, plot=False, eps=eps, min_samples=min_samples
            )

            unnormalized_data, _ = unnormalize(
                result_dbscan.data, None, data_min, data_max
            )

            plotter = ClusterPlotter(unnormalized_data, result_dbscan.labels, None)  # No cluster centers
            plot_filename = os.path.join(plot_dir, f"dbscan_{year_range}_eps{eps}_min{min_samples}.png")
            title = f"DBSCAN: years = {year_range}, eps = {eps}, min_samples = {min_samples}"
            plotter.clusters_2d_plot(title, plot_filename, show_centers=False)


<>:19: SyntaxWarning: invalid escape sequence '\I'
<>:19: SyntaxWarning: invalid escape sequence '\I'
C:\Users\fionu\AppData\Local\Temp\ipykernel_23496\4201603990.py:19: SyntaxWarning: invalid escape sequence '\I'
  plot_dir = "Images\IAC_plots\DBSCAN_2d_5years"


{'2005-2009': {'geo': ['../input/stat_Master_05_geo_s1.crs', '../input/stat_Master_06_geo_s1.crs', '../input/stat_Master_07_geo_s1.crs', '../input/stat_Master_08_geo_s1.crs', '../input/stat_Master_09_geo_s1.crs'], 'gto': ['../input/stat_Master_05_gto_s1.crs', '../input/stat_Master_06_gto_s1.crs', '../input/stat_Master_07_gto_s1.crs', '../input/stat_Master_08_gto_s1.crs', '../input/stat_Master_09_gto_s1.crs'], 'fol': ['../input/stat_Master_05_fol_s1.crs', '../input/stat_Master_06_fol_s1.crs', '../input/stat_Master_07_fol_s1.crs', '../input/stat_Master_08_fol_s1.crs', '../input/stat_Master_09_fol_s1.crs']}}

Running DBSCAN for Year Range: 2005-2009
Runtime for dbscan_clustering: 0.212400 seconds
Plot saved as: Images\IAC_plots\DBSCAN_2d_5years\dbscan_2005-2009_eps0.015_min27.png


# PROOF plots in > 2D

## KMeans

In [44]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cluster_data_pca_08072025 as cdp
from cluster_data_pca_08072025 import run_clustering, normalize_data
from kmeans import k_means
import cluster_plotter

# --- Style settings ---
plt.rcParams.update({
    "font.size": 14,
    "axes.labelsize": 16,
    "axes.titlesize": 18,
    "legend.fontsize": 14,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14
})

# --- Setup ---
year_range = "2005-2008"
data_set = {year_range: np.arange(2005, 2009)}
plot_dir = "Images/IAC_plots/KMeans_2Dplus"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

# --- Load and prepare data ---
binned = cdp.bin_data_for_clustering(data_set, print_res=False)
data_obj, _ = binned[0]

# --- Full feature set for clustering (9D) ---
X_all = np.vstack([
    data_obj.ecc,
    data_obj.sem_maj,
    data_obj.inc,
    data_obj.raan,
    data_obj.perigee,
    data_obj.true_lat,
    data_obj.mean_motion,
    data_obj.mag_obj,
    data_obj.diameter
]).T

norm_X_all, X_min, X_max = normalize_data(X_all)

# --- Subset of features just for plotting (6D) ---
X_plot = np.vstack([
    data_obj.mean_motion,   # mean motion
    data_obj.ecc,           # eccentricity
    data_obj.inc,           # inclination
    data_obj.raan,          # RAAN
    data_obj.true_lat,      # argument of latitude
    data_obj.mag_obj        # magnitude
]).T

# --- K-Means parameters ---
k_values = [5, 6, 7]

# Extreme contrast color palette
extreme_colors = [
    "#FF000094", "#3DC53D", "#4F4FF3", '#FFFF00', "#A148A1", '#00FFFF',
    '#FF8000', '#8000FF', "#F157A4", '#00FF80', '#804000', '#000000',
    '#808080', '#404040'
]

# Map features to LaTeX labels with units
axis_labels = {
    "mean motion": r"$n$ [rev/day]",
    "ecc": r"$e$",
    "inc": r"$i$ [°]",
    "raan": r"$\Omega$ [°]",
    "arg lat": r"$\lambda$ [°]",
    "mag": r"$m$"
}

for k in k_values:
    # --- Run clustering in 9D ---
    clustering, duration, n_clusters, pts_per_cluster, _ = run_clustering(
        k_means, f"K-Means (k={k})",
        norm_X_all, X_min, X_max, k, init='kmeans++'
    )
    labels = clustering.labels

    # --- Build dataframe for plotting (6D + labels) ---
    df = pd.DataFrame(X_plot, columns=[
        "mean motion", "ecc", "inc", "raan", "arg lat", "mag"
    ])
    df['cluster'] = labels.astype(int)

    # --- Pairplot with extreme contrast colors ---
    pp = sns.pairplot(
        df, hue='cluster', diag_kind='kde',
        plot_kws={'alpha': 0.8, 's': 20, 'edgecolor': 'black', 'linewidth': 0.5},
        corner=True,
        hue_order=sorted(df['cluster'].unique()),
        palette=extreme_colors
    )

    # Replace axis labels with LaTeX
    for ax in pp.axes.flatten():
        if ax is not None:
            xlabel = ax.get_xlabel()
            ylabel = ax.get_ylabel()
            if xlabel in axis_labels:
                ax.set_xlabel(axis_labels[xlabel])
            if ylabel in axis_labels:
                ax.set_ylabel(axis_labels[ylabel])

    # Force labels also on diagonal plots
    for i in range(len(pp.diag_axes)):
        ax = pp.diag_axes[i]
        var = pp.x_vars[i]
        if var in axis_labels:
            ax.set_xlabel(axis_labels[var])
            ax.set_ylabel(axis_labels[var])

    # Fix the first diagonal label explicitly
    if len(pp.diag_axes) > 0:
        first_diag_ax = pp.diag_axes[0]
        first_var = pp.x_vars[0]
        if first_var in axis_labels:
            first_diag_ax.set_ylabel(axis_labels[first_var])

    # Enlarge legend & markers with black borders
    for lh in pp._legend.legend_handles:
        lh.set_markersize(20)
        lh.set_alpha(1.0)
        lh.set_markeredgecolor('black')
        lh.set_markeredgewidth(1.5)

    pp._legend.set_title("Cluster", prop={'size': 28})
    for text in pp._legend.get_texts():
        text.set_fontsize(28)

    pp._legend.set_bbox_to_anchor((1.15, 1))

    # Title
    pp.fig.suptitle(
        f"K-Means 9D→6D Clusters {year_range} (k={k})",
        y=1.02, fontsize=20
    )

    fname = f"pairplot_{year_range}_k{k}.png"
    pp.savefig(os.path.join(plot_dir, fname), bbox_inches="tight", dpi=300, facecolor='white')
    plt.close(pp.fig)


Runtime for k_means: 0.062593 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

Runtime for k_means: 0.128625 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

Runtime for k_means: 0.044971 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

## DBSCAN

In [43]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cluster_data_pca_08072025 as cdp
from cluster_data_pca_08072025 import run_clustering, normalize_data
from DBSCAN import dbscan_clustering
import cluster_plotter

# --- Style settings ---
plt.rcParams.update({
    "font.size": 14,       # base font
    "axes.labelsize": 16,  # axis labels
    "axes.titlesize": 18,  # subplot titles
    "legend.fontsize": 14, # legend font
    "xtick.labelsize": 14,
    "ytick.labelsize": 14
})

year_range = "2005-2008"
data_set = {year_range: np.arange(2005, 2009)}
plot_dir = "Images/IAC_plots/DBSCAN_2Dplus"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

# --- Load and prepare data ---
binned = cdp.bin_data_for_clustering(data_set, print_res=False)
data_obj, _ = binned[0]

# --- Full feature set for clustering (9D) ---
X_all = np.vstack([
    data_obj.ecc,
    data_obj.sem_maj,
    data_obj.inc,
    data_obj.raan,
    data_obj.perigee,
    data_obj.true_lat,
    data_obj.mean_motion,
    data_obj.mag_obj,
    data_obj.diameter
]).T

norm_X_all, X_min, X_max = normalize_data(X_all)

# --- Subset of features just for plotting (6D) ---
X_plot = np.vstack([
    data_obj.mean_motion,
    data_obj.ecc,
    data_obj.inc,
    data_obj.raan,
    data_obj.true_lat,
    data_obj.mag_obj
]).T

# --- DBSCAN parameters ---
eps_values = [0.2, 0.12, 0.13, 0.0135, 0.014]
min_samples_values = [10]

for eps in eps_values:
    for min_samples in min_samples_values: 
        # --- Run clustering in 9D ---
        clustering, duration, n_clusters, pts_per_cluster, _ = run_clustering(
            dbscan_clustering, "DBSCAN", norm_X_all, X_min, X_max,
            eps=eps, min_samples=min_samples
        )
        labels = clustering.labels

        # --- Map features to LaTeX labels with units ---
        axis_labels = {
            "mean motion": r"$n$ [rev/day]",
            "ecc": r"$e$",
            "inc": r"$i$ [°]",
            "raan": r"$\Omega$ [°]",
            "arg lat": r"$\lambda$ [°]",
            "mag": r"$m$"
        }

        df = pd.DataFrame(X_plot, columns=[
            "mean motion", "ecc", "inc", "raan", "arg lat", "mag"
        ])
        df['cluster'] = labels.astype(int)

        # --- Extreme contrast color scheme ---
        extreme_colors = [
            "#FF000094",  # Pure Red
            "#3DC53D",    # Pure Green  
            "#4F4FF3",    # Pure Blue
            '#FFFF00',    # Pure Yellow
            "#A148A1",    # Pure Magenta
            '#00FFFF',    # Pure Cyan
            '#FF8000',    # Bright Orange
            '#8000FF',    # Bright Purple
            "#F157A4",    # Hot Pink
            '#00FF80',    # Bright Teal
            '#804000',    # Dark Brown
            '#000000',    # Black
            '#808080',    # Medium Gray
            '#404040',    # Dark Gray
        ]

        sns.set_palette(extreme_colors)

        # --- Pairplot with extreme contrast colors ---
        pp = sns.pairplot(
            df, hue='cluster', diag_kind='kde',
            plot_kws={'alpha': 0.8, 's': 20, 'edgecolor': 'black', 'linewidth': 0.5}, 
            corner=True,
            hue_order=sorted(df['cluster'].unique()),
            palette=extreme_colors
        )

        # Replace axis labels with LaTeX
        for ax in pp.axes.flatten():
            if ax is not None:
                xlabel = ax.get_xlabel()
                ylabel = ax.get_ylabel()
                if xlabel in axis_labels:
                    ax.set_xlabel(axis_labels[xlabel])
                if ylabel in axis_labels:
                    ax.set_ylabel(axis_labels[ylabel])

        # Force labels also on diagonal plots (some are missing with corner=True)
        for i, var in enumerate(pp.x_vars):
            if i < len(pp.axes) and i < len(pp.axes[i]):
                ax = pp.axes[i, i]
                if ax is not None and var in axis_labels:
                    ax.set_xlabel(axis_labels[var])
                    ax.set_ylabel(axis_labels[var])

        # --- Enlarge and reposition legend ---
        for lh in pp._legend.legend_handles:
            lh.set_markersize(20)
            lh.set_alpha(1.0)
            lh.set_markeredgewidth(1)
        pp._legend.set_title("Cluster", prop={'size': 28})
        for text in pp._legend.get_texts():
            text.set_fontsize(28)
        pp._legend.set_bbox_to_anchor((1.15, 1))

        # Title
        pp.fig.suptitle(
            f"DBSCAN 9D→6D Clusters {year_range} (eps={eps}, ms={min_samples})",
            y=1.02, fontsize=20
        )

        # Save
        fname = f"pairplot_{year_range}_eps{eps}_ms{min_samples}.png"
        pp.savefig(os.path.join(plot_dir, fname), bbox_inches="tight", dpi=300, facecolor='white')
        plt.close(pp.fig)


Runtime for dbscan_clustering: 0.457117 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (8), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (8), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (8), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (8), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (8), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

Runtime for dbscan_clustering: 0.247186 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: 
The palette list has fewer values (14) than needed (19) and will cycle, which may produce an uninterpretable plot.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: 
The palette list has fewer values (14) than needed (19) and will cycle, which may produce an uninterpretable plot.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: 
The palette list has fewer values (14) than needed (19) and will cycle, which may produce an uninterpretable plot.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: 
The palette list has fewer values (14) than needed (19) and will cycle, which may produce an uninterpretable plot.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning

Runtime for dbscan_clustering: 0.558848 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: 
The palette list has fewer values (14) than needed (20) and will cycle, which may produce an uninterpretable plot.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: 
The palette list has fewer values (14) than needed (20) and will cycle, which may produce an uninterpretable plot.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: 
The palette list has fewer values (14) than needed (20) and will cycle, which may produce an uninterpretable plot.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: 
The palette list has fewer values (14) than needed (20) and will cycle, which may produce an uninterpretable plot.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning

Runtime for dbscan_clustering: 0.160887 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

Runtime for dbscan_clustering: 0.149621 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (1), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

# Oberservation plots in 2D

## KMeans

In [1]:
import numpy as np
import os
import cluster_data
import cluster_plotter
from cluster_data import run_clustering, normalize_data, unnormalize
from cluster_plotter import ClusterPlotter
from kmeans import k_means

# Define observed files
uncorr_obs_files = {
    year: f"ogs{year}01_12_det.ele_ucorr" if year != 2002 else f"ogs{year}08_12_det.ele_ucorr"
    for year in range(2002, 2024) if year != 2018  # Exclude 2018 if missing
}

# Year ranges
IAC_range = {"2005-2008": np.arange(2005, 2009)}
year_ranges = IAC_range

# Bin observed data
binned_data = cluster_data.bin_observed_data(uncorr_obs_files, year_ranges, print_res=False)

# Output directory
plot_dir = "Images/IAC_plots/KMeans_2d_observed"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

# Run clustering and plot
for cluster_data, year_range in binned_data:
    print(f"\nRunning K-Means for Year Range: {year_range}")
    
    data_array = np.array([cluster_data.inc, cluster_data.raan]).T
    normalized_data, data_min, data_max = normalize_data(data_array)
    
    for k in [5, 6, 7, 8, 9]:
        result_kmeans, _, _, _, _ = run_clustering(
            k_means, "K-Means", normalized_data, data_min, data_max, k, plot=False
        )

        unnormalized_data, cluster_centers = unnormalize(
            result_kmeans.data, result_kmeans.cluster_centers, data_min, data_max
        )
        plotter = ClusterPlotter(unnormalized_data, result_kmeans.labels, cluster_centers)
        plotter.clusters_2d_plot(
            f"K-Means: years = {year_range}, k = {k}",
            os.path.join(plot_dir, f"kmeans_{year_range}_k{k}.png")
        )



Running K-Means for Year Range: 2005-2008
Runtime for k_means: 0.032113 seconds
Plot saved as: Images/IAC_plots/KMeans_2d_observed\kmeans_2005-2008_k5.png
Runtime for k_means: 0.047455 seconds
Plot saved as: Images/IAC_plots/KMeans_2d_observed\kmeans_2005-2008_k6.png
Runtime for k_means: 0.137462 seconds
Plot saved as: Images/IAC_plots/KMeans_2d_observed\kmeans_2005-2008_k7.png
Runtime for k_means: 0.018192 seconds
Plot saved as: Images/IAC_plots/KMeans_2d_observed\kmeans_2005-2008_k8.png
Runtime for k_means: 0.099716 seconds
Plot saved as: Images/IAC_plots/KMeans_2d_observed\kmeans_2005-2008_k9.png


## DBSCAN

In [3]:
import numpy as np
import os
import cluster_data
from cluster_data import normalize_data, unnormalize
from cluster_plotter import ClusterPlotter
from DBSCAN import dbscan_clustering  # Assuming you have a DBSCAN implementation

# Define observed files
uncorr_obs_files = {
    year: f"ogs{year}01_12_det.ele_ucorr" if year != 2002 else f"ogs{year}08_12_det.ele_ucorr"
    for year in range(2002, 2024) if year != 2018  # Exclude 2018 if missing
}

# Use running ranges
IAC_range = {"2005-2008": np.arange(2005, 2009)}
year_ranges = IAC_range

# Bin the observed data
binned_data = cluster_data.bin_observed_data(uncorr_obs_files, year_ranges, print_res=False)

# Output directory for plots
plot_dir = "Images/IAC_plots/DBSCAN_2d_observed"
os.makedirs(plot_dir, exist_ok=True)

for cluster_data, year_range in binned_data:
    data_array = np.array([cluster_data.inc, cluster_data.raan]).T
    normalized_data, data_min, data_max = normalize_data(data_array)
    
    eps_values = [0.02, 0.01, 0.015, 0.05]
    min_samples_values = [10, 15, 20, 25]
    
    for eps in eps_values:
        for min_samples in min_samples_values:
            result_dbscan = dbscan_clustering(
                normalized_data, eps=eps, min_samples=min_samples
            )

            unnormalized_data, cluster_centers = unnormalize(
                result_dbscan.data, result_dbscan.cluster_centers, data_min, data_max
            )
            
            plotter = ClusterPlotter(unnormalized_data, result_dbscan.labels, cluster_centers)
            plotter.clusters_2d_plot(
                f"DBSCAN: years = {year_range}, eps = {eps}, min_samples = {min_samples}",
                os.path.join(plot_dir, f"dbscan_{year_range}_eps{eps}_min{min_samples}.png")
            )


Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.02_min10.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.02_min15.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.02_min20.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.02_min25.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.01_min10.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.01_min15.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.01_min20.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.01_min25.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.015_min10.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.015_min15.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\dbscan_2005-2008_eps0.015_min20.png
Plot saved as: Images/IAC_plots/DBSCAN_2d_observed\

# Observed data > 2D

## KMeans

In [5]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cluster_data_pca_08072025 as cdp
from cluster_data_pca_08072025 import run_clustering, normalize_data
from kmeans import k_means
import cluster_plotter

# --- Style settings ---
plt.rcParams.update({
    "font.size": 14,
    "axes.labelsize": 16,
    "axes.titlesize": 18,
    "legend.fontsize": 14,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14
})

# --- Setup ---
year_range = "2005-2008"
plot_dir = "Images/IAC_plots/KMeans_2Dplus_observed"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

# --- Load and prepare observed data ---
uncorr_obs_files = {
    year: f"ogs{year}01_12_det.ele_ucorr" if year != 2002 else f"ogs{year}08_12_det.ele_ucorr"
    for year in range(2002, 2024) if year != 2018
}

year_ranges = {year_range: np.arange(2005, 2009)}
binned_data = cdp.bin_observed_data(uncorr_obs_files, year_ranges, print_res=False)

# --- K-Means parameters ---
k_values = [5, 6, 7]

# Extreme contrast color palette
extreme_colors = [
    "#FF000094", "#3DC53D", "#4F4FF3", '#FFFF00', "#A148A1", '#00FFFF',
    '#FF8000', '#8000FF', "#F157A4", '#00FF80', '#804000', '#000000',
    '#808080', '#404040'
]

# Map features to LaTeX labels with units
axis_labels = {
    "mean motion": r"$n$ [rev/day]",
    "ecc": r"$e$",
    "inc": r"$i$ [°]",
    "raan": r"$\Omega$ [°]",
    "arg lat": r"$\lambda$ [°]",
    "mag": r"$m$"
}

# --- Run clustering and create pairplots ---
for cluster_data, year_range in binned_data:
    print(f"\nRunning 9D K-Means for Year Range: {year_range}")

    # --- Full feature set for clustering (9D) ---
    X_all = np.vstack([
        cluster_data.ecc,
        cluster_data.sem_maj,
        cluster_data.inc,
        cluster_data.raan,
        cluster_data.perigee,
        cluster_data.true_lat,
        cluster_data.mean_motion,
        cluster_data.mag_obj,
        cluster_data.diameter
    ]).T

    norm_X_all, X_min, X_max = normalize_data(X_all)

    # --- Subset of features just for plotting (6D) ---
    X_plot = np.vstack([
        cluster_data.mean_motion,
        cluster_data.ecc,
        cluster_data.inc,
        cluster_data.raan,
        cluster_data.true_lat,
        cluster_data.mag_obj
    ]).T

    print(cluster_data.true_lat)

    for k in k_values:
        # --- Run K-Means in 9D ---
        clustering, duration, n_clusters, pts_per_cluster, _ = run_clustering(
            k_means, f"K-Means (k={k})",
            norm_X_all, X_min, X_max, k, init='kmeans++'
        )
        labels = clustering.labels

        # --- Build dataframe for pairplot ---
        df = pd.DataFrame(X_plot, columns=[
            "mean motion", "ecc", "inc", "raan", "arg lat", "mag"
        ])
        df['cluster'] = labels.astype(int)

        # --- Pairplot ---
        pp = sns.pairplot(
            df, hue='cluster', diag_kind='kde',
            plot_kws={'alpha': 0.8, 's': 20, 'edgecolor': 'black', 'linewidth': 0.5},
            corner=True,
            hue_order=sorted(df['cluster'].unique()),
            palette=extreme_colors
        )

        # Replace axis labels with LaTeX
        for ax in pp.axes.flatten():
            if ax is not None:
                xlabel = ax.get_xlabel()
                ylabel = ax.get_ylabel()
                if xlabel in axis_labels:
                    ax.set_xlabel(axis_labels[xlabel])
                if ylabel in axis_labels:
                    ax.set_ylabel(axis_labels[ylabel])

        # Force labels also on diagonal plots
        for i in range(len(pp.diag_axes)):
            ax = pp.diag_axes[i]
            var = pp.x_vars[i]
            if var in axis_labels:
                ax.set_xlabel(axis_labels[var])
                ax.set_ylabel(axis_labels[var])

        # Fix the first diagonal label explicitly
        if len(pp.diag_axes) > 0:
            first_diag_ax = pp.diag_axes[0]
            first_var = pp.x_vars[0]
            if first_var in axis_labels:
                first_diag_ax.set_ylabel(axis_labels[first_var])

        # Enlarge legend & markers with black borders
        for lh in pp._legend.legend_handles:
            lh.set_markersize(20)
            lh.set_alpha(1.0)
            lh.set_markeredgecolor('black')
            lh.set_markeredgewidth(1.5)

        pp._legend.set_title("Cluster", prop={'size': 28})
        for text in pp._legend.get_texts():
            text.set_fontsize(28)
        pp._legend.set_bbox_to_anchor((1.15, 1))

        # Title
        pp.fig.suptitle(
            f"K-Means 9D→6D Clusters {year_range} (k={k})",
            y=1.02, fontsize=20
        )

        fname = f"pairplot_{year_range}_k{k}.png"
        pp.savefig(os.path.join(plot_dir, fname), bbox_inches="tight", dpi=300, facecolor='white')
        plt.close(pp.fig)


Running 9D K-Means for Year Range: 2005-2008
[ -4.3311885  -8.4727279 -15.2347987 ... -21.5007717  -4.360807
  -4.385722 ]
Runtime for k_means: 0.049823 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (5), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

Runtime for k_means: 0.231096 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (6), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

Runtime for k_means: 0.119040 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (7), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

## DBSCAN

In [4]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cluster_data_pca_08072025 as cdp
from cluster_data_pca_08072025 import run_clustering, normalize_data
from DBSCAN import dbscan_clustering
import cluster_plotter

# --- Style settings ---
plt.rcParams.update({
    "font.size": 14,
    "axes.labelsize": 16,
    "axes.titlesize": 18,
    "legend.fontsize": 14,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14
})

# --- Setup ---
year_range = "2005-2008"
plot_dir = "Images/IAC_plots/DBSCAN_2Dplus_observed"
os.makedirs(plot_dir, exist_ok=True)
cluster_plotter.clear_directory(plot_dir)

# --- Load and prepare observed data ---
uncorr_obs_files = {
    year: f"ogs{year}01_12_det.ele_ucorr" if year != 2002 else f"ogs{year}08_12_det.ele_ucorr"
    for year in range(2002, 2024) if year != 2018
}
year_ranges = {year_range: np.arange(2005, 2009)}
binned_data = cdp.bin_observed_data(uncorr_obs_files, year_ranges, print_res=False)

# --- DBSCAN parameters ---
eps_values = [0.1, 0.11, 0.13]
min_samples_values = [7, 10, 20]

# Extreme contrast color palette
extreme_colors = [
    "#FF000094", "#3DC53D", "#4F4FF3", '#FFFF00', "#A148A1", '#00FFFF',
    '#FF8000', '#8000FF', "#F157A4", '#00FF80', '#804000', '#000000',
    '#808080', '#404040'
]

axis_labels = {
    "mean motion": r"$n$ [rev/day]",
    "ecc": r"$e$",
    "inc": r"$i$ [°]",
    "raan": r"$\Omega$ [°]",
    "arg lat": r"$\lambda$ [°]",
    "mag": r"$m$"
}

# --- Run DBSCAN on observed data ---
for cluster_data, year_range in binned_data:
    print(f"\nRunning DBSCAN 9D for Year Range: {year_range}")

    # --- Full feature set for clustering (9D) ---
    X_all = np.vstack([
        cluster_data.ecc,
        cluster_data.sem_maj,
        cluster_data.inc,
        cluster_data.raan,
        cluster_data.perigee,
        cluster_data.true_lat,
        cluster_data.mean_motion,
        cluster_data.mag_obj,
        cluster_data.diameter
    ]).T

    norm_X_all, X_min, X_max = normalize_data(X_all)

    # --- Subset of features for 6D plotting ---
    X_plot = np.vstack([
        cluster_data.mean_motion,
        cluster_data.ecc,
        cluster_data.inc,
        cluster_data.raan,
        cluster_data.true_lat,
        cluster_data.mag_obj
    ]).T

    for eps in eps_values:
        for min_samples in min_samples_values:
            # --- Run DBSCAN in 9D ---
            clustering, duration, n_clusters, pts_per_cluster, _ = run_clustering(
                dbscan_clustering, "DBSCAN", norm_X_all, X_min, X_max,
                eps=eps, min_samples=min_samples
            )
            labels = clustering.labels

            # --- Build dataframe for pairplot ---
            df = pd.DataFrame(X_plot, columns=[
                "mean motion", "ecc", "inc", "raan", "arg lat", "mag"
            ])
            df['cluster'] = labels.astype(int)

            # --- Pairplot ---
            pp = sns.pairplot(
                df, hue='cluster', diag_kind='kde',
                plot_kws={'alpha': 0.8, 's': 20, 'edgecolor': 'black', 'linewidth': 0.5}, 
                corner=True,
                hue_order=sorted(df['cluster'].unique()),
                palette=extreme_colors
            )

            # Replace axis labels with LaTeX
            for ax in pp.axes.flatten():
                if ax is not None:
                    xlabel = ax.get_xlabel()
                    ylabel = ax.get_ylabel()
                    if xlabel in axis_labels:
                        ax.set_xlabel(axis_labels[xlabel])
                    if ylabel in axis_labels:
                        ax.set_ylabel(axis_labels[ylabel])

            # Force labels on diagonal
            for i, var in enumerate(pp.x_vars):
                if i < len(pp.axes) and i < len(pp.axes[i]):
                    ax = pp.axes[i, i]
                    if ax is not None and var in axis_labels:
                        ax.set_xlabel(axis_labels[var])
                        ax.set_ylabel(axis_labels[var])

            # --- Enlarge and reposition legend ---
            for lh in pp._legend.legend_handles:
                lh.set_markersize(20)
                lh.set_alpha(1.0)
                lh.set_markeredgewidth(1)
            pp._legend.set_title("Cluster", prop={'size': 28})
            for text in pp._legend.get_texts():
                text.set_fontsize(28)
            pp._legend.set_bbox_to_anchor((1.15, 1))

            # Title
            pp.fig.suptitle(
                f"DBSCAN 9D→6D Clusters {year_range} (eps={eps}, ms={min_samples})",
                y=1.02, fontsize=20
            )

            # Save
            fname = f"pairplot_{year_range}_eps{eps}_ms{min_samples}.png"
            pp.savefig(os.path.join(plot_dir, fname), bbox_inches="tight", dpi=300, facecolor='white')
            plt.close(pp.fig)



Running DBSCAN 9D for Year Range: 2005-2008
Runtime for dbscan_clustering: 0.297205 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (13), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (13), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (13), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (13), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (13), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\

Runtime for dbscan_clustering: 0.445210 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (10), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (10), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (10), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (10), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (10), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\

Runtime for dbscan_clustering: 0.455695 seconds


...

c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (4), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (4), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (4), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (4), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (4), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

Runtime for dbscan_clustering: 0.292420 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco

Runtime for dbscan_clustering: 0.225299 seconds


c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaconda3\Lib\site-packages\seaborn\axisgrid.py:1513: UserWarning: The palette list has more values (14) than needed (3), which may not be intended.
  func(x=vector, **plot_kwargs)
c:\Users\fionu\anaco